# PhonePe-Pulse-Data-Visualisation-And-Exploration:
In this section of the project, we are going to 

* Extract the data of transactions and users of PhonePe
* Upload the data to MySQL database.

In [1]:
import pandas as pd
import mysql.connector as sql
import streamlit as st
import plotly.express as px
import os
import json
from streamlit_option_menu import option_menu
from PIL import Image

C:\Users\sanju>git clone https://github.com/PhonePe/pulse.git

Cloning into 'pulse'...
remote: Enumerating objects: 15229, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 15229 (delta 3), reused 1 (delta 1), pack-reused 15202
Receiving objects: 100% (15229/15229), 21.95 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (7745/7745), done.
Updating files: 100% (7921/7921), done.

### 1. Extracting the aggregated transactions data:

In [2]:
path1 = "C:/Users/sanju/OneDrive/Desktop/Projects/PhonePe/pulse/data/aggregated/transaction/country/india/state/"
agg_trans_list = os.listdir(path1)

columns1 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [],
            'Transaction_amount': []}
for state in agg_trans_list:
    cur_state = path1 + state + "/"
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            A = json.load(data)
            
            for i in A['data']['transactionData']:
                name = i['name']
                count = i['paymentInstruments'][0]['count']
                amount = i['paymentInstruments'][0]['amount']
                columns1['Transaction_type'].append(name)
                columns1['Transaction_count'].append(count)
                columns1['Transaction_amount'].append(amount)
                columns1['State'].append(state)
                columns1['Year'].append(year)
                columns1['Quarter'].append(int(file.strip('.json')))
                
agg_trans_df = pd.DataFrame(columns1)

In [3]:
# Taking a look at the unique state values:
agg_trans_df['State'].unique()

array(['andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
       'assam', 'bihar', 'chandigarh', 'chhattisgarh',
       'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
       'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
       'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
       'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
       'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
       'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
       'uttarakhand', 'west-bengal'], dtype=object)

In [4]:
# Formatting the States field in the desired format:
agg_trans_df['State']=agg_trans_df['State'].str.title()
agg_trans_df['State']=agg_trans_df['State'].str.replace("-"," ")
agg_trans_df['State']=agg_trans_df['State'].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu')

### 2. Extracting the aggregated user data:

In [5]:
path2 = "C:/Users/sanju/OneDrive/Desktop/Projects/PhonePe/pulse/data/aggregated/user/country/india/state/"

agg_user_list = os.listdir(path2)

columns2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [],
            'Percentage': []}
for state in agg_user_list:
    cur_state = path2 + state + "/"
    agg_year_list = os.listdir(cur_state)
    
    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            B = json.load(data)
            try:
                for i in B["data"]["usersByDevice"]:
                    brand_name = i["brand"]
                    counts = i["count"]
                    percents = i["percentage"]
                    columns2["Brands"].append(brand_name)
                    columns2["Count"].append(counts)
                    columns2["Percentage"].append(percents)
                    columns2["State"].append(state)
                    columns2["Year"].append(year)
                    columns2["Quarter"].append(int(file.strip('.json')))
            except:
                pass
agg_user_df = pd.DataFrame(columns2)

In [6]:
# Formatting the States field in the desired format:
agg_user_df['State']=agg_user_df['State'].str.title()
agg_user_df['State']=agg_user_df['State'].str.replace("-"," ")
agg_user_df['State']=agg_user_df['State'].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu')

### 3. Extracting the Map transactions data:

In [7]:
path3 = "C:/Users/sanju/OneDrive/Desktop/Projects/PhonePe/pulse/data/map/transaction/hover/country/india/state/"

map_trans_list = os.listdir(path3)

columns3 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [],
            'Amount': []}

for state in map_trans_list:
    cur_state = path3 + state + "/"
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            C = json.load(data)
            
            for i in C["data"]["hoverDataList"]:
                district = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns3["District"].append(district)
                columns3["Count"].append(count)
                columns3["Amount"].append(amount)
                columns3['State'].append(state)
                columns3['Year'].append(year)
                columns3['Quarter'].append(int(file.strip('.json')))
                
map_trans_df = pd.DataFrame(columns3)  

In [8]:
# Formatting the States field in the desired format:
map_trans_df['State']=map_trans_df['State'].str.title()
map_trans_df['District']=map_trans_df['District'].str.title()
map_trans_df['State']=map_trans_df['State'].str.replace("-"," ")
map_trans_df['State']=map_trans_df['State'].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu')

### 4. Extracting the Map user data:

In [9]:
path4 = "C:/Users/sanju/OneDrive/Desktop/Projects/PhonePe/pulse/data/map/user/hover/country/india/state/"

map_user_list = os.listdir(path4)

columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
            "RegisteredUser": [], "AppOpens": []}

for state in map_user_list:
    cur_state = path4 + state + "/"
    map_year_list = os.listdir(cur_state)
    
    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)
        
        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            D = json.load(data)
            
            for i in D["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appOpens = i[1]['appOpens']
                columns4["District"].append(district)
                columns4["RegisteredUser"].append(registereduser)
                columns4["AppOpens"].append(appOpens)
                columns4['State'].append(state)
                columns4['Year'].append(year)
                columns4['Quarter'].append(int(file.strip('.json')))
                
map_user_df = pd.DataFrame(columns4)

In [10]:
# Formatting the States field in the desired format:
map_user_df['State']=map_trans_df['State'].str.title()
map_user_df['District']=map_trans_df['District'].str.title()
map_user_df['State']=map_trans_df['State'].str.replace("-"," ")
map_user_df['State']=map_trans_df['State'].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu')

In [11]:
# Checking if there are null values:
map_user_df.isnull().sum()
map_user_df.tail()

# There are a few nulls in State field.

,State,Year,Quarter,District,RegisteredUser,AppOpens
18295,West Bengal,2024,1,Darjiling District,1671140,13679202
18296,NaN,2024,1,NaN,1056537,30502875
18297,NaN,2024,1,NaN,1655920,18688405
18298,NaN,2024,1,NaN,1183956,33270738
18299,NaN,2024,1,NaN,686254,31485682


In [12]:
# Based on research, these districts are present in the state of West Bengal.
# Hence, we will fill these NaN with "West Bengal"

map_user_df = map_user_df.fillna("West Bengal")

In [13]:
# the field is confirmed to have no null values now:
map_user_df.isna().sum()

State             0
Year              0
Quarter           0
District          0
RegisteredUser    0
AppOpens          0
dtype: int64

### 5. Extracting the Top Transactions data:

In [14]:
path5 = "C:/Users/sanju/OneDrive/Desktop/Projects/PhonePe/pulse/data/top/transaction/country/india/state/"

top_trans_list = os.listdir(path5)
columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],
            'Transaction_amount': []}

for state in top_trans_list:
    cur_state = path5 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            E = json.load(data)
            
            for i in E['data']['pincodes']:
                name = i['entityName']
                count = i['metric']['count']
                amount = i['metric']['amount']
                columns5['Pincode'].append(name)
                columns5['Transaction_count'].append(count)
                columns5['Transaction_amount'].append(amount)
                columns5['State'].append(state)
                columns5['Year'].append(year)
                columns5['Quarter'].append(int(file.strip('.json')))
top_trans_df = pd.DataFrame(columns5)
            

In [15]:
# Formatting the States field in the desired format:
top_trans_df['State']=top_trans_df['State'].str.title()
top_trans_df['State']=top_trans_df['State'].str.replace("-"," ")
top_trans_df['State']=top_trans_df['State'].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu')

### 6. Extracting the Top Users data:

In [16]:
path6 = "C:/Users/sanju/OneDrive/Desktop/Projects/PhonePe/pulse/data/top/user/country/india/state/"
top_user_list = os.listdir(path6)
columns6 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
            'RegisteredUsers': []}

for state in top_user_list:
    cur_state = path6 + state + "/"
    top_year_list = os.listdir(cur_state)
    
    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)
        
        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            F = json.load(data)
            
            for i in F['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                columns6['Pincode'].append(name)
                columns6['RegisteredUsers'].append(registeredUsers)
                columns6['State'].append(state)
                columns6['Year'].append(year)
                columns6['Quarter'].append(int(file.strip('.json')))
top_user_df = pd.DataFrame(columns6)         

In [17]:
# Formatting the States field in the desired format:
top_user_df['State']=top_user_df['State'].str.title()
top_user_df['State']=top_user_df['State'].str.replace("-"," ")
top_user_df['State']=top_user_df['State'].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu')

In [18]:
# Presence of null in dataset:
top_user_df.isnull().sum()
top_trans_df[top_trans_df.isnull().any(axis=1)]

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
4301,Ladakh,2019,4,None,2014,1.009866e+07
4343,Ladakh,2020,4,None,13717,3.671160e+07


In [19]:
# print the shape of the data:
print('Shape of Aggregated Transactions data :', agg_trans_df.shape)
print('Shape of Aggregated Users data :', agg_user_df.shape)
print('Shape of Map Transactions data :', map_trans_df.shape)
print('Shape of Map Users data :', map_user_df.shape)
print('Shape of Top Transactions data :', top_trans_df.shape)
print('Shape of Top Users data :', top_user_df.shape)

Shape of Aggregated Transactions data : (4494, 6)
Shape of Aggregated Users data : (6732, 6)
Shape of Map Transactions data : (18296, 6)
Shape of Map Users data : (18300, 6)
Shape of Top Transactions data : (8924, 6)
Shape of Top Users data : (8925, 5)


### Converting the dataframes to csv files:

In [20]:
agg_trans_df.to_csv('agg_trans.csv',index=False)
agg_user_df.to_csv('agg_user.csv',index=False)
map_trans_df.to_csv('map_trans.csv',index=False)
map_user_df.to_csv('map_user.csv',index=False)
top_trans_df.to_csv('top_trans.csv',index=False)
top_user_df.to_csv('top_user.csv',index=False)

### Connecting to your MySQL database

In [21]:
mydb = sql.connect(host="localhost",
                   user="root",
                   password="Sanc3098",
                   database= "phonepe"
                  )
mycursor = mydb.cursor(buffered=True)

# Using the phonepe database:
mycursor.execute("USE phonepe;")

### Creating the tables to insert the respective datasets:

#### Table for Aggregated transactions data

In [22]:
mycursor.execute('''Create table if not exists agg_trans (State varchar(100), 
                 Year int, Quarter int, Transaction_type varchar(100), Transaction_count bigint, 
                 Transaction_amount double)''')

for i,row in agg_trans_df.iterrows():
    #here %S means string values 
    sql = "INSERT INTO agg_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    mydb.commit()  

#### Table for Aggregated Users data

In [23]:
mycursor.execute('''Create table if not exists agg_user (State varchar(100), 
                 Year int, Quarter int, Brands varchar(100), User_count bigint, User_percentage double)''')

for i,row in agg_user_df.iterrows():
    sql = "INSERT INTO agg_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

#### Table for Map transactions data

In [24]:
mycursor.execute('''Create table if not exists map_trans (State varchar(100), 
                 Year int, Quarter int, District varchar(100), Transaction_count bigint, 
                 Transaction_amount double)''')

for i,row in map_trans_df.iterrows():
    sql = "INSERT INTO map_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

#### Table for Map Users data

In [25]:
mycursor.execute('''Create table if not exists map_user (State varchar(100), 
                 Year int, Quarter int, District varchar(100), Registered_users bigint, 
                 App_opens bigint)''')

for i,row in map_user_df.iterrows():
    sql = "INSERT INTO map_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

#### Table for Top transactions data

In [26]:
mycursor.execute('''create table if not exists top_trans (State varchar(100), 
                 Year int, Quarter int, Pincode int, Transaction_count bigint, Transaction_amount double)''')

for i,row in top_trans_df.iterrows():
    sql = "INSERT INTO top_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

#### Table for Top Users data

In [27]:
mycursor.execute('''create table top_user (State varchar(100), 
                 Year int, Quarter int, Pincode int, Registered_users bigint)''')

for i,row in top_user_df.iterrows():
    sql = "INSERT INTO top_user VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [28]:
mycursor.execute("show tables")
mycursor.fetchall()

[('agg_trans',),
 ('agg_user',),
 ('map_trans',),
 ('map_user',),
 ('top_trans',),
 ('top_user',)]